# Compute downward C flux from UVP data

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import linregress
%matplotlib inline

In [27]:
dir='/Users/leo/Documents/MOSAIC_UVP/'

In [28]:
# import data
data = pd.read_csv(dir+'data/export_detailed_20211011_08_50/export_detailed_20211011_08_50_PAR_Aggregated.tsv', sep = "\t", encoding= 'unicode_escape')
# only keep the data spectra (45 size classes)
datasub = data.iloc[:,6:51]

In [4]:
data_matrix = datasub.values
data_matrix

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [5]:
def pasvar(minor, maxor, pas = 2**(1/3)):
    
    """
        function [ves,new,med]=pasvar(minor,maxor,pas)
    %           [ves,new,med]=pasvar(minor,maxor,pas)
    %
    % Avec :
    % Construction d'un vecteur à pas variable pour les histogrammes (ves)
    % Construction d'un vecteur mesurant la taille des intervals entre les valeurs extremes (new)
    % Construction d'un vecteur avec les valeurs médianes des pas (med)
    %
    % Use pas = 2^(1/3) for the ECOTAXA size classes (double the size of the classes evry 4 classes)
    """

    ves = []
    ves.append(minor)
    n = 1000
    
    for i in range(1,n):
        ves.append(pas*ves[i-1])
        if ves[i] > maxor:
            break
    
    h = len(ves)
    
    sous = np.array(ves[1:])
    new = sous - ves[:-1]
    
    med = []
    for i in range(0,h-1):
        med.append((ves[i]+ves[i+1])/2)

    return(np.array(ves), new, np.array(med))

In [6]:
classe, tailles, DSE = pasvar(0.001, 30, 2**(1/3))

In [7]:
def nb2fluxECOpart(X, mini = 0.001, maxi = 30, pas = 2**(1/3), A = 12.5, B = 3.81):
    
    """
    
        % Estimate carbon flux from particle size distribution (ECOpart)
    %
    %           [flux,classes,DSE,sizeclasses]=nb2fluxECOpart(X,mini,maxi,pas,A,B)
    % Input:
    %   X : Concentration of particles (Nb/l per size classes) => Flo : a matrix/dataframe
    %   mini : Minimum size for particles (1 micron) .. in mm
    %   maxi : Maximum size for particles (26 cm) .. in mm
    %   pas : Geometric progression for the size classes (2^1/3 pour ECOpart)
    %   A : Coefficient from Guidi et al., 2008 (default 12.5 for Carbon)
    %   B : Exponent from Guidi et al., 2008    (defaut 3.81 for Carbon)
    % 
    % Output:
    %   flux : Matrix of fluxes en mg.m^(-2).d^(-1) : Tot Flux (classes 25 to 32)
    %   classes : Size classes in mm
    %   DSE : Mean of the size classes in mm
    %   sizeclasses : Size of the size bins in mm
    
    """
    
    nrow = X.shape[0]
    ncol = X.shape[1]
    flux = np.zeros((nrow, ncol))
    
    # direct estimates of class in mm
    classe, tailles, DSE = pasvar(mini, maxi, pas)
    classes = np.zeros((len(classe)-1,2))
    classes[:,0] = classe[:-1]
    classes[:,1] = classe[1:]

    # calcul du flux
    flu = A*DSE**B # Formule pour le calcul du flu basee sur les minimisation sur les pieges a sediment
    
    # Fin de la Troisieme partie
    flux = np.zeros(data_matrix.shape)
    for i in range(0, nrow):
        flux[i,:]=data_matrix[i,:]*flu
    
    return(flux, classes, DSE, tailles)

In [8]:
def slope_particle(nb, tailles, DSE):
    # % Calcul d'un profil de pente en fonction du nombre de particules par
    # % classe de taille et du diametre moyen des particules
    # % 
    # %   [slope]=slope_particle(nb,size_classe,DSE)
    # %
    # % Input:    - nb: Matrice du nombre de particules #/L
    # %           - size_classe: Vecteur avec la taille des intervals (in mm)
    # %           - DSE: Vecteur contenant la taille des particules
    # % 
    # % Output:   - slope: Vecteur contenant les valeur de pente

    #mini = 0.001
    #maxi = 30
    #pas = 2**(1/3)
    #classe, tailles, DSE = pasvar(mini, maxi, pas)
    
    nrow = nb.shape[0]
    ncol = nb.shape[1]
    
    nb_norm = nb/np.tile(tailles, (nrow, 1))
    log_y = np.log(nb_norm)
    log_x = np.log(DSE)
    
    result_slope = []
    
    for i in range(nrow):
        Y = log_y[i,:]
        X = log_x
        h = np.isfinite(Y)
        h = np.concatenate(np.where(h==True))
        Y = Y[h]
        X = X[h]
        if len(Y) >=5:
            slope, intercept, r_value, p_value, std_err = linregress(X,Y)
            result_slope.append(slope)
        else:
            slope = np.nan
            result_slope.append(slope)
            
    return(np.array(result_slope))

In [9]:
flux, classes, DSE, tailles = nb2fluxECOpart(data_matrix)

In [10]:
def compute_total_flux(data, flux):
    # restricted to classes 25 to 32
    t_flux = np.nansum(flux[:,24:32], axis = 1)
    tmp = data.loc[:,['Profile', 'Rawfilename', 'yyyy-mm-dd hh:mm', 'Project', 'Depth [m]', 'Sampled volume [L]']]
    tmp['Total flux [mg m-2 d-1]'] = t_flux
    
    return(tmp)

In [11]:
total_flux = compute_total_flux(data, flux)
total_flux

,Profile,Rawfilename,yyyy-mm-dd hh:mm,Project,Depth [m],Sampled volume [L],Total flux [mg m-2 d-1]
0,ps122_1_2_68,HDR20191004082819,2019-10-04 08:28:19,uvp5_sn204_mosaic_primary_filtered,2.5,196.56,2.042380
1,ps122_1_2_68,HDR20191004082819,2019-10-04 08:28:19,uvp5_sn204_mosaic_primary_filtered,7.5,234.36,1.962594
2,ps122_1_2_68,HDR20191004082819,2019-10-04 08:28:19,uvp5_sn204_mosaic_primary_filtered,12.5,210.60,3.885001
3,ps122_1_2_68,HDR20191004082819,2019-10-04 08:28:19,uvp5_sn204_mosaic_primary_filtered,17.5,184.68,5.064934
4,ps122_1_2_68,HDR20191004082819,2019-10-04 08:28:19,uvp5_sn204_mosaic_primary_filtered,22.5,150.12,2.655590
...,...,...,...,...,...,...,...
25771,ps122_5_59_72,HDR20200818062715,2020-08-18 06:27:15,uvp5_sn204_mosaic_primary_filtered,992.5,247.32,0.157842
25772,ps122_5_59_72,HDR20200818062715,2020-08-18 06:27:15,uvp5_sn204_mosaic_primary_filtered,997.5,278.64,0.147273
25773,ps122_5_59_72,HDR20200818062715,2020-08-18 06:27:15,uvp5_sn204_mosaic_primary_filtered,1002.5,223.56,0.208886
25774,ps122_5_59_72,HDR20200818062715,2020-08-18 06:27:15,uvp5_sn204_mosaic_primary_filtered,1007.5,236.52,0.338285


In [24]:
# add slopes
slopes = slope_particle(data_matrix[:,20:], tailles[20:], DSE[20:])
total_flux['slope']= slopes

/var/folders/0g/k89gmbc176318q62lgz26vrc0000gn/T/ipykernel_27998/3360819244.py:22: RuntimeWarning: invalid value encountered in log
  log_y = np.log(nb_norm)


In [29]:
total_flux['time'] = pd.to_datetime(total_flux['yyyy-mm-dd hh:mm'])
total_flux.drop(columns = 'yyyy-mm-dd hh:mm', inplace=True)
total_flux.to_csv(dir+'data/mosaic_flux_slope.csv', index = False)